In [1]:
import pandas as pd

data = pd.read_csv('Crop_Dataset.csv')

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, train_size=0.8)
validation_data, test_data = train_test_split(test_data, train_size=0.5)

train_data.shape, test_data.shape, validation_data.shape

((1760, 12), (220, 12), (220, 12))

In [2]:
n_classes = len(data['Label_Encoded'].unique())
n_classes

22

In [3]:
import torch

device = "mps"
print(f"Using {device} device")

Using mps device


In [4]:
from torch.utils.data import Dataset
from torch.nn.functional import normalize

class CustomDataset(Dataset):
    def __init__(self, data):
        self.inputs = data.iloc[:, 0:-2].values.astype('float32')
        
        # for i in range(self.inputs.shape[1]):
        #     self.inputs[:, i] = normalize(torch.tensor(self.inputs[:, i].reshape(-1, 1)), dim=0).reshape(-1)
        
        for i in range(self.inputs.shape[1]):
            max_val = self.inputs[:, i].max()
            min_val = self.inputs[:, i].min()
            self.inputs[:, i] = (self.inputs[:, i] - min_val) / (max_val - min_val)
        
        self.outputs = data.iloc[:, -1].values.astype('int')
        
        self.inputs = torch.tensor(self.inputs, device=device)
        self.outputs = torch.tensor(self.outputs, device=device)

    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index]

    def __len__(self):
        return len(self.inputs)

In [5]:
batch_size = 128

train_data_loader = torch.utils.data.DataLoader(CustomDataset(train_data), batch_size=batch_size, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(CustomDataset(test_data), batch_size=batch_size, shuffle=True)
validation_data_loader = torch.utils.data.DataLoader(CustomDataset(validation_data), batch_size=batch_size, shuffle=True)

In [6]:
# print shapes and samples from the dataset
for X, y in train_data_loader:
    print("Shape of X:", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    print("Sample X: ", X[0])
    print("Sample y: ", y[0])
    break

Shape of X: torch.Size([128, 10])
Shape of y:  torch.Size([128]) torch.int64
Sample X:  tensor([0.0429, 0.4786, 0.0500, 0.4103, 0.6119, 0.6478, 0.1243, 0.2044, 0.3362,
        0.3644], device='mps:0')
Sample y:  tensor(8, device='mps:0')


In [7]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 22),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=22, bias=True)
  )
)


In [9]:
# testing predictions

_X = torch.rand(1, 10, device=device)
logits = model(_X)
predicted_prob = nn.Softmax(dim=1)(logits)
y_pred = predicted_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([19], device='mps:0')


In [10]:
learning_rate = 1e-2
batch_size = 64
epochs = 500

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
import wandb

configs = {
    "learning_rate": learning_rate,
    "architecture": "ANN",
    "dataset": "Crop",
    "epochs": epochs,
    "batch_size": batch_size
}

wandb.init(
    project="Intellihack_1",
    config=configs
)

wandb: Currently logged in as: lakshith. Use `wandb login --relogin` to force relogin


In [12]:
def train_loop(dataloader):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), batch * batch_size + len(X)
        
        wandb.log({
            "Loss": loss
        })
        print(f"\rloss: {loss:>7f}  [{current:>5d}/{size:>5d}]", end="")

In [13]:
def test_loop(dataloader):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    wandb.log({
        "Test Loss": test_loss,
        "Test Accuracy": correct
    })
    
    print("Test: \nAccuracy: ", correct, "\nAvg loss: ", test_loss)

In [14]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    print("")
    train_loop(train_data_loader)
    test_loop(test_data_loader)

print("\n\n\nValidation Data: ")
test_loop(validation_data_loader)

Epoch 1
-------------------------------
loss: 3.092394  [  928/ 1760]Test: 
Accuracy:  0.08636363636363636 
Avg loss:  3.082542061805725
Epoch 2
-------------------------------

loss: 3.071829  [  928/ 1760]Test: 
Accuracy:  0.08181818181818182 
Avg loss:  3.074304938316345
Epoch 3
-------------------------------

loss: 3.052243  [  928/ 1760]Test: 
Accuracy:  0.08181818181818182 
Avg loss:  3.064993143081665
Epoch 4
-------------------------------

loss: 3.048591  [  928/ 1760]Test: 
Accuracy:  0.09545454545454546 
Avg loss:  3.0603822469711304
Epoch 5
-------------------------------

loss: 3.058642  [  928/ 1760]Test: 
Accuracy:  0.11818181818181818 
Avg loss:  3.0514646768569946
Epoch 6
-------------------------------

loss: 3.047478  [  928/ 1760]Test: 
Accuracy:  0.14545454545454545 
Avg loss:  3.0442930459976196
Epoch 7
-------------------------------
loss: 3.027323  [  928/ 1760]Test: 
Accuracy:  0.14545454545454545 
Avg loss:  3.0354868173599243
Epoch 8
------------------------